In [15]:
import pandas as pd
import numpy as np
import re
import nltk

In [16]:
df = pd.read_csv("/Users/kumarsanjeev/desktop/LabelledData (1).txt",sep=",,,",header=None ,names=['question','label'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [17]:
df.head()

,question,label
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [18]:
df.shape

(1483, 2)

In [19]:
set(df['label'])

{' affirmation', ' unknown', ' what', ' when', ' who'}

In [20]:
df['label']=df['label'].str.strip()

In [21]:
set(df['label'])

{'affirmation', 'unknown', 'what', 'when', 'who'}

In [22]:
df['question'].head()

0    how did serfdom develop in and then leave russ...
1    what films featured the character popeye doyle ? 
2    how can i find a list of celebrities ' real na...
3    what fowl grabs the spotlight after the chines...
4                     what is the full form of .com ? 
Name: question, dtype: object

In [23]:
df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [24]:
df['question'].head()

0    how did serfdom develop in and then leave russ...
1     what films featured the character popeye doyle  
2    how can i find a list of celebrities  real nam...
3    what fowl grabs the spotlight after the chines...
4                       what is the full form of com  
Name: question, dtype: object

In [25]:
((df['question'].iloc[0].strip()))

'how did serfdom develop in and then leave russia'

In [26]:
print(df['label'].value_counts())

what           609
who            402
unknown        272
affirmation    104
when            96
Name: label, dtype: int64


In [27]:
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle as pkl
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
from sklearn.model_selection import  train_test_split

In [28]:
stemmer = SnowballStemmer('english').stem
def stem_tokenize(text):
    return [stemmer(i) for i in word_tokenize(text)]

In [29]:
vectorizer = TfidfVectorizer(analyzer='word',lowercase=True,tokenizer=stem_tokenize)
X_train = vectorizer.fit_transform(df['question'].values)

In [30]:
labels = df['label']
df['question']


0       how did serfdom develop in and then leave russ...
1        what films featured the character popeye doyle  
2       how can i find a list of celebrities  real nam...
3       what fowl grabs the spotlight after the chines...
4                          what is the full form of com  
5       what contemptible scoundrel stole the cork fro...
6       what team did baseball s st louis browns become  
7                         what is the oldest profession  
8                                what are liver enzymes  
9       name the scarfaced bounty hunter of the old we...
10                          when was ozzy osbourne born  
11        why do heavier objects travel downhill faster  
12                     who was the pride of the yankees  
13                                    who killed gandhi  
14      what is considered the costliest disaster the ...
15      what sprawling us state boasts the most airpor...
16      what did the only repealed amendment to the us...
17      how ma

In [31]:
'''TEST_SPLIT=0.33

# split the data into a training set and a test set
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
test_samples = int(TEST_SPLIT * X_train.shape[0])

x_train = X_train[:-test_samples]
y_train = labels[:-test_samples]
x_test = X_train[-test_samples:]
y_test = labels[-test_samples:]'''
x_train,x_test ,y_train,y_test = train_test_split(X_train ,labels , random_state = 4 ,test_size = 0.3)

In [32]:
# train model
clf = MultinomialNB(0.1)
clf.fit(x_train,y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [24]:
# evaluate the model on test data
pred = clf.predict(x_test)
print(classification_report(pred,y_test))
print("Accuracy : {}".format(clf.score(x_test,y_test)))

             precision    recall  f1-score   support

affirmation       0.72      0.95      0.82        22
    unknown       0.78      0.73      0.75        90
       what       0.88      0.83      0.85       197
       when       0.58      0.55      0.56        20
        who       0.81      0.87      0.84       116

avg / total       0.82      0.82      0.82       445

Accuracy : 0.8157303370786517


In [42]:
# predict new data 
dat=vectorizer.transform([" Who is Trump"])
yy = clf.predict(dat)

In [43]:
print('Question-Type: {}'.format(yy))

Question-Type: ['who']
